In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports for langchain and Chroma and plotly

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [3]:
!C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\python.exe -m pip install nbformat





You should consider upgrading via the 'C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
# price is a factor for our company, so we're going to use a low cost model

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [7]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, UnstructuredHTMLLoader

# Paths for each data type
pdf_folder = "data_ingestion/mosdac_data/documents"
web_folder = "data_ingestion/mosdac_data/web_pages"

# --- Load PDFs ---
pdf_loader = DirectoryLoader(
    path=pdf_folder,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)
pdf_docs = pdf_loader.load()

# --- Load TXT ---


txt_loader = DirectoryLoader(
    path=web_folder,
    glob="**/*.txt",
    loader_cls=lambda path: TextLoader(path, encoding="utf-8")
)
txt_docs = txt_loader.load()


# --- Load HTML ---
html_loader = DirectoryLoader(
    path=web_folder,
    glob="**/*.html",
    loader_cls=UnstructuredHTMLLoader
)
html_docs = html_loader.load()

# Combine all docs
documents = pdf_docs + txt_docs + html_docs

print(f"✅ Total documents loaded: {len(documents)}")


Multiple definitions in dictionary at byte 0x30f8cb for key /Im1027
Multiple definitions in dictionary at byte 0x30f8dc for key /Im1027


✅ Total documents loaded: 1485


In [8]:
from langchain.text_splitter import CharacterTextSplitter

# Try different chunk size & overlap combinations
chunk_configs = [
    {"size": 800, "overlap": 150},
    {"size": 1000, "overlap": 200},
    {"size": 1500, "overlap": 300},
    {"size": 2000, "overlap": 400}
]

for cfg in chunk_configs:
    print(f"\n=== chunk_size={cfg['size']}, chunk_overlap={cfg['overlap']} ===")
    
    text_splitter = CharacterTextSplitter(
        chunk_size=cfg["size"],
        chunk_overlap=cfg["overlap"]
    )
    chunks = text_splitter.split_documents(documents)
    
    print(f"Total chunks: {len(chunks)}")
    print(f"Example chunk (first 300 chars):\n{chunks[0].page_content[:300]}")


Created a chunk of size 3630, which is longer than the specified 800
Created a chunk of size 1110, which is longer than the specified 800
Created a chunk of size 3393, which is longer than the specified 800
Created a chunk of size 2513, which is longer than the specified 800
Created a chunk of size 2505, which is longer than the specified 800
Created a chunk of size 2503, which is longer than the specified 800
Created a chunk of size 2516, which is longer than the specified 800
Created a chunk of size 2180, which is longer than the specified 800
Created a chunk of size 2508, which is longer than the specified 800
Created a chunk of size 3590, which is longer than the specified 800
Created a chunk of size 3203, which is longer than the specified 800
Created a chunk of size 2501, which is longer than the specified 800
Created a chunk of size 2512, which is longer than the specified 800
Created a chunk of size 2509, which is longer than the specified 800
Created a chunk of size 2511, whic


=== chunk_size=800, chunk_overlap=150 ===


Created a chunk of size 2518, which is longer than the specified 800
Created a chunk of size 1608, which is longer than the specified 800
Created a chunk of size 2498, which is longer than the specified 800
Created a chunk of size 2139, which is longer than the specified 800
Created a chunk of size 3649, which is longer than the specified 800
Created a chunk of size 2519, which is longer than the specified 800
Created a chunk of size 2539, which is longer than the specified 800
Created a chunk of size 3547, which is longer than the specified 800
Created a chunk of size 2517, which is longer than the specified 800
Created a chunk of size 2512, which is longer than the specified 800
Created a chunk of size 2514, which is longer than the specified 800
Created a chunk of size 2509, which is longer than the specified 800
Created a chunk of size 2578, which is longer than the specified 800
Created a chunk of size 2526, which is longer than the specified 800
Created a chunk of size 2514, whic

Total chunks: 1866
Example chunk (first 300 chars):
GLOBAL ANALYSED OCEAN SURFACE WIND PRODUCTS 
 
 
Description 
The analysed winds have been generated at 0.5 
0×0.5 0 interval over the global oceans. For the generation of 
these analysed winds products, we have used (Oceansat-2 SCATterometer) OSCAT data for daily Analysed 
Wind Vector (AWV) and com

=== chunk_size=1000, chunk_overlap=200 ===


Created a chunk of size 3348, which is longer than the specified 1000
Created a chunk of size 4160, which is longer than the specified 1000
Created a chunk of size 4044, which is longer than the specified 1000
Created a chunk of size 4135, which is longer than the specified 1000
Created a chunk of size 3620, which is longer than the specified 1000
Created a chunk of size 6095, which is longer than the specified 1000
Created a chunk of size 3351, which is longer than the specified 1000
Created a chunk of size 4128, which is longer than the specified 1000
Created a chunk of size 4160, which is longer than the specified 1000
Created a chunk of size 4132, which is longer than the specified 1000
Created a chunk of size 3930, which is longer than the specified 1000
Created a chunk of size 3633, which is longer than the specified 1000
Created a chunk of size 5317, which is longer than the specified 1000
Created a chunk of size 2841, which is longer than the specified 1000
Created a chunk of s

Total chunks: 1848
Example chunk (first 300 chars):
GLOBAL ANALYSED OCEAN SURFACE WIND PRODUCTS 
 
 
Description 
The analysed winds have been generated at 0.5 
0×0.5 0 interval over the global oceans. For the generation of 
these analysed winds products, we have used (Oceansat-2 SCATterometer) OSCAT data for daily Analysed 
Wind Vector (AWV) and com

=== chunk_size=1500, chunk_overlap=300 ===


Created a chunk of size 2516, which is longer than the specified 1500
Created a chunk of size 2180, which is longer than the specified 1500
Created a chunk of size 2508, which is longer than the specified 1500
Created a chunk of size 3590, which is longer than the specified 1500
Created a chunk of size 3203, which is longer than the specified 1500
Created a chunk of size 2501, which is longer than the specified 1500
Created a chunk of size 2512, which is longer than the specified 1500
Created a chunk of size 2509, which is longer than the specified 1500
Created a chunk of size 2511, which is longer than the specified 1500
Created a chunk of size 3724, which is longer than the specified 1500
Created a chunk of size 2518, which is longer than the specified 1500
Created a chunk of size 1608, which is longer than the specified 1500
Created a chunk of size 2498, which is longer than the specified 1500
Created a chunk of size 2139, which is longer than the specified 1500
Created a chunk of s

Total chunks: 1812
Example chunk (first 300 chars):
GLOBAL ANALYSED OCEAN SURFACE WIND PRODUCTS 
 
 
Description 
The analysed winds have been generated at 0.5 
0×0.5 0 interval over the global oceans. For the generation of 
these analysed winds products, we have used (Oceansat-2 SCATterometer) OSCAT data for daily Analysed 
Wind Vector (AWV) and com

=== chunk_size=2000, chunk_overlap=400 ===


Created a chunk of size 4135, which is longer than the specified 2000
Created a chunk of size 3620, which is longer than the specified 2000
Created a chunk of size 6095, which is longer than the specified 2000
Created a chunk of size 3351, which is longer than the specified 2000
Created a chunk of size 4128, which is longer than the specified 2000
Created a chunk of size 4160, which is longer than the specified 2000
Created a chunk of size 4132, which is longer than the specified 2000
Created a chunk of size 3930, which is longer than the specified 2000
Created a chunk of size 3633, which is longer than the specified 2000
Created a chunk of size 5317, which is longer than the specified 2000
Created a chunk of size 2841, which is longer than the specified 2000
Created a chunk of size 4186, which is longer than the specified 2000
Created a chunk of size 2506, which is longer than the specified 2000
Created a chunk of size 2327, which is longer than the specified 2000
Created a chunk of s

Total chunks: 1802
Example chunk (first 300 chars):
GLOBAL ANALYSED OCEAN SURFACE WIND PRODUCTS 
 
 
Description 
The analysed winds have been generated at 0.5 
0×0.5 0 interval over the global oceans. For the generation of 
these analysed winds products, we have used (Oceansat-2 SCATterometer) OSCAT data for daily Analysed 
Wind Vector (AWV) and com


In [9]:
len(chunks)


1802

In [10]:
print(chunks[0].metadata)


{'producer': 'Nitro PDF PrimoPDF', 'creator': 'PrimoPDF http://www.primopdf.com', 'creationdate': '2012-03-20T14:49:18-05:30', 'moddate': '2012-03-20T14:49:18-05:30', 'title': 'Microsoft Word - Analysed-Winds', 'author': 'admin', 'source': 'data_ingestion\\mosdac_data\\documents\\Analysed-Winds.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}


In [11]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [12]:
# Check if a Chroma Datastore already exists - if so, delete the collection to start from scratch

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [14]:
from langchain.vectorstores import Chroma
import math

# Estimated safe token limit for your embedding model (OpenAI text-embedding-3-large)
MAX_TOKENS_PER_BATCH = 8000  # Adjust if using another model

def get_safe_batch_size(chunks, max_tokens_per_batch=MAX_TOKENS_PER_BATCH):
    avg_tokens_per_chunk = sum(len(doc.page_content.split()) for doc in chunks) / len(chunks)
    safe_batch_size = max(1, int(max_tokens_per_batch // avg_tokens_per_chunk))
    return min(safe_batch_size, 200)  # cap to 200 so you don't overload API

# Calculate safe batch size
batch_size = get_safe_batch_size(chunks)

print(f"⚙️  Auto-selected batch size: {batch_size}")

# Create Chroma vectorstore
vectorstore = Chroma(
    persist_directory=db_name,
    embedding_function=embeddings
)

# Upload in batches
total_chunks = len(chunks)
for i in range(0, total_chunks, batch_size):
    batch = chunks[i:i+batch_size]
    est_tokens = sum(len(doc.page_content.split()) for doc in batch)
    vectorstore.add_documents(batch)
    print(f"✅ Uploaded batch {i//batch_size + 1}/{math.ceil(total_chunks/batch_size)} - ~{est_tokens} tokens")

# Final vector count
print(f"\n🎉 Vectorstore now has {vectorstore._collection.count()} documents")


⚙️  Auto-selected batch size: 32
✅ Uploaded batch 1/57 - ~8433 tokens
✅ Uploaded batch 2/57 - ~6842 tokens
✅ Uploaded batch 3/57 - ~7019 tokens
✅ Uploaded batch 4/57 - ~8525 tokens
✅ Uploaded batch 5/57 - ~8852 tokens
✅ Uploaded batch 6/57 - ~10573 tokens
✅ Uploaded batch 7/57 - ~11660 tokens
✅ Uploaded batch 8/57 - ~9979 tokens
✅ Uploaded batch 9/57 - ~9699 tokens
✅ Uploaded batch 10/57 - ~11732 tokens
✅ Uploaded batch 11/57 - ~10830 tokens
✅ Uploaded batch 12/57 - ~8739 tokens
✅ Uploaded batch 13/57 - ~3887 tokens
✅ Uploaded batch 14/57 - ~5635 tokens
✅ Uploaded batch 15/57 - ~10259 tokens
✅ Uploaded batch 16/57 - ~12428 tokens
✅ Uploaded batch 17/57 - ~10007 tokens
✅ Uploaded batch 18/57 - ~8939 tokens
✅ Uploaded batch 19/57 - ~9284 tokens
✅ Uploaded batch 20/57 - ~11000 tokens
✅ Uploaded batch 21/57 - ~10400 tokens
✅ Uploaded batch 22/57 - ~11739 tokens
✅ Uploaded batch 23/57 - ~8068 tokens
✅ Uploaded batch 24/57 - ~8947 tokens
✅ Uploaded batch 25/57 - ~6975 tokens
✅ Uploaded batch

In [16]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")


The vectors have 1,536 dimensions


In [17]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']


In [19]:
# Extract doc_type from file name (e.g., 'Analysed-Winds.pdf' → 'analysed')
doc_types = [metadata['source'].split('\\')[-1].split('-')[0].lower() for metadata in result['metadatas']]

# Define color map
color_map = {
    'analysed': 'blue',
    'winds': 'green',
    # Add more based on your actual file naming
}

# Assign colors to each document type
colors = [color_map.get(doc_type, 'gray') for doc_type in doc_types]


In [ ]:
import umap
import matplotlib.pyplot as plt

# Reduce to 2D
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
vectors_2d = reducer.fit_transform(vectors)

# Plot
plt.figure(figsize=(10, 6))
for i, point in enumerate(vectors_2d):
    plt.scatter(point[0], point[1], color=colors[i], alpha=0.6)

plt.title("Vector Embeddings (UMAP Visualization)", fontsize=14)
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.grid(True)
plt.show()


KeyboardInterrupt: 

In [ ]:
from sklearn.manifold import TSNE
import plotly.graph_objects as go
#

# Apply t-SNE to reduce to 3D
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()


In [24]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
import gradio as gr

# This assumes MODEL and vectorstore are already defined in your notebook.
# For example:
# MODEL = "gpt-4o-mini"
# llm = ChatOpenAI(temperature=0.7, model_name=MODEL)
# memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
# retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


# Define a custom prompt template for the RAG chain when documents are found.
rag_prompt_template = """Your name is MOSDAC AI, a helpful assistant.
Answer the question based on the provided documents and chat history.
If you find the answer in the documents, provide a detailed and comprehensive response.
If the documents do not contain the answer, you should still attempt to answer the question using your general knowledge in a detailed and plausible way.

Here is the context from the documents:
{context}

Here is the chat history:
{chat_history}

Question: {question}
"""
RAG_PROMPT = PromptTemplate.from_template(rag_prompt_template)

# Define a prompt template for the general knowledge fallback when no documents are relevant.
fallback_prompt_template = """You are MOSDAC AI, a helpful assistant.
Answer the user's question in a detailed and plausible manner, using your general knowledge.
Aim for a response of at least 6-7 lines for simple queries and more for complex ones.
Question: {question}
"""
FALLBACK_PROMPT = PromptTemplate.from_template(fallback_prompt_template)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={'prompt': RAG_PROMPT}
)

def is_relevant(question, docs):
    """Check if any doc is actually relevant to the question."""
    if not docs:
        return False
    check = llm.invoke(
        f"Is this document relevant to the question? Answer Yes or No.\\n\\nQuestion: {question}\\nDocument: {docs[0].page_content}"
    )
    return "Yes" in check.content

def smart_bot(message, history):
    docs = retriever.get_relevant_documents(message)
    
    # Use the relevance check to decide the mode
    if is_relevant(message, docs):
        # RAG mode: Use the specialized RAG chain
        result = rag_chain.invoke({"question": message})
        return result['answer']
    else:
        # Fallback mode: Use a direct LLM call with a custom prompt to encourage detailed, plausible answers
        fallback_chain = FALLBACK_PROMPT | llm
        result = fallback_chain.invoke({"question": message})
        return result.content

gr.ChatInterface(
    fn=smart_bot,
    title="ASTROGEO",
    theme="soft"
).launch(share=True)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://6b5e5c94bdf72c2bf9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
query = "What is analysed wind data?"
response = conversation_chain.run(query)
print(response)


NameError: name 'conversation_chain' is not defined

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
docs = retriever.get_relevant_documents("What is the exact naming convention for ASCAT L2B files in MOSDAC for April 2023?")
for doc in docs:
    print(doc.page_content)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_8620\1875945586.py:2: LangChainDeprecationWarning:

The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.



1 | P a g e  
 
 
MRG/EPSA/SAC/Dec2020/Version1.0 
 
 
 
 
 
 
 
 
Meteorological and Oceanographic Satellite Data Archival Centre (MOSDAC) 
Data Dissemination Guidelines 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Space Applications Centre (ISRO) 
Ahmedabad, INDIA
SC1/DP/L4FORMAT-DOC/V1.1/JUL2017 
 
 
 
 
 
 
 
SCATSAT-1 Level 4 Data Products 
Format Document 
 
 
 
 
 
 
 
 
 
 
 
 
By 
Scatsat1 DP Team 
 
 
 
 
Microwave Data Processing Division 
Signal and Image Processing Group 
Space Applications Centre 
Ahmedabad
INSAT
-
3D 
Incremental 
ATBD
 
(MMDRPS)
, 
November 2018
 
 
            
 
 
 
 
 
1
 
 
 
(MMDRPS Incremental ATBD)
 
November, 2018
